In [ ]:
pip install deep-translator langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=9c41e392b7d7f127b4914f300f01c924bc7df60ff610d7f3e9130bf22a16e7c0
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
import pandas as pd
import re
from deep_translator import GoogleTranslator
from langdetect import detect, DetectorFactory

df=pd.read_csv('reviews.csv')
df.tail()

,listing_id,id,date,reviewer_id,reviewer_name,comments
1205942,1364616759503109256,1368127111176342942,2025-03-02,368146996,Clémentine,"Très bon séjour chez Juan, le logement était p..."
1205943,1365007887657848515,1370244234430465527,2025-03-05,283312107,Hernan Enrique,Recomendable para una persona que quiere hacer...
1205944,1365007887657848515,1373818429800403177,2025-03-10,184521825,Carlos,"Es una Host súper amable, fue atenta en todo m..."
1205945,1365635357360238932,1368103217874537823,2025-03-02,313257078,Yuet,很友善的房东，推荐
1205946,1366534590465861868,1368063220606082280,2025-03-02,369293146,Hamid,Very friendly host and staff and Great locatio...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117151 entries, 0 to 1205946
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1117151 non-null  int64 
 1   id             1117151 non-null  int64 
 2   date           1117151 non-null  object
 3   reviewer_id    1117151 non-null  int64 
 4   reviewer_name  1117148 non-null  object
 5   comments       1117151 non-null  object
dtypes: int64(3), object(3)
memory usage: 59.7+ MB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117151 entries, 0 to 1205946
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1117151 non-null  int64 
 1   id             1117151 non-null  int64 
 2   date           1117151 non-null  object
 3   reviewer_id    1117151 non-null  int64 
 4   reviewer_name  1117148 non-null  object
 5   comments       1117151 non-null  object
dtypes: int64(3), object(3)
memory usage: 59.7+ MB


In [ ]:
DetectorFactory.seed = 0

# Función para detectar idioma
def detect_language(text):
    try:
        if pd.isna(text) or text.strip() == "":
            return None
        return detect(text)
    except Exception:
        return None

# Cargar archivo CSV en modo lectura por lotes
chunk_size = 100  # Número de reseñas por lote
input_file = "reviews.csv"
output_file = "clean_reviews.csv"
first_chunk = True

for chunk in pd.read_csv(input_file, chunksize=chunk_size):
    chunk["language"] = chunk["comments"].apply(detect_language)

    chunk.to_csv(output_file, mode="a", header=first_chunk, index=False)
    first_chunk = False  # Solo escribir el encabezado en la primera iteración
    #print(f"Procesado {chunk_size} reseñas...")

print("✅ Traducción completada y guardada en", output_file)

In [ ]:
df_1=pd.read_csv('clean_reviews.csv')
df_1.tail()

**Esta línea de código solo se hace la primera vez. Toma unas cuantas horas en traducir. Para seguir con la traducción cuando se interrumpe, hacerlo con la de más adelante.**

In [ ]:
# Función para traducir usando la columna de idioma
def translate_review(text, lang):
    try:
        if pd.isna(text) or pd.isna(lang):
            return None
        if lang == "en":  # Si ya está en inglés, no traducir
            return text
        return GoogleTranslator(source=lang, target="en").translate(text)
    except Exception:
        return None

chunk_size = 1000
input_file = "clean_reviews.csv"
output_file = "translated_reviews.csv"
first_chunk = True

for chunk in pd.read_csv(input_file, chunksize=chunk_size):
    chunk["translated_review"] = chunk.apply(lambda row: translate_review(row["comments"], row["language"]), axis=1)

    chunk.to_csv(output_file, mode="a", header=first_chunk, index=False)
    first_chunk = False

print("✅ Traducción completada y guardada en", output_file)

In [ ]:
# Función para traducir usando la columna de idioma
def translate_review(text, lang):
    try:
        if pd.isna(text) or pd.isna(lang):
            return None
        if lang == "en":
            return text
        return GoogleTranslator(source=lang, target="en").translate(text)
    except Exception:
        return None

**Seguir con este código a partir de la segunda vez que se traduce.**

In [ ]:
chunk_size = 1000
input_file = "clean_reviews.csv"
output_file = "translated_reviews.csv"

try:
    translated_df = pd.read_csv(output_file)
    translated_ids = set(translated_df["id"])
    first_chunk = False
except FileNotFoundError:
    translated_ids = set()
    first_chunk = True

for chunk in pd.read_csv(input_file, chunksize=chunk_size):
    chunk = chunk[~chunk["id"].isin(translated_ids)] # Filtrar solo las reseñas traducidas

    if chunk.empty:
        continue

    chunk["translated_review"] = chunk.apply(lambda row: translate_review(row["comments"], row["language"]), axis=1)

    chunk.to_csv(output_file, mode="a", header=first_chunk, index=False)
    first_chunk = False

print("✅ Traducción completada y guardada en", output_file)

✅ Traducción completada y guardada en translated_reviews.csv


In [ ]:
df_t=pd.read_csv('translated_reviews.csv')
df_t.tail()

In [ ]:
df_t